In [ ]:
import gradio as gr
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from io import BytesIO
from skimage.color import rgb2lab, lab2rgb
import math, cmath, io

In [2]:
#converts an image into grayscale
def image_to_matrix(image):
    grayscale = image.convert("L")
    mat = []
    for y in range(grayscale.height):
        row = []
        for x in range(grayscale.width):
            row.append(grayscale.getpixel((x,y)))
        mat.append(row)
    return mat

In [3]:
# conert an matrix back into a image
def matrix_to_image(matrix):
    M, N = len(matrix), len(matrix[0])
    img = Image.new("L", (N, M))
    for x in range(M):
        for y in range(N):
            pixelBrightness = int(min(max(matrix[x][y], 0), 255))
            img.putpixel((y, x), pixelBrightness)
    return img

In [4]:
#perform DFT
def DFT2d(matrix):
    M = len(matrix)
    N = len(matrix[0])
    F = []
    for n in range(M):
        row = []
        for m in range(N):
            row.append(0)
        F.append(row)
    
    for u in range(M):
        for v in range(N):
            sum_c = 0
            for x in range(M):
                for y in range(N):
                    angle = -2 * math.pi * ((u * x / M) + (v * y / N))
                    sum_c += matrix[x][y] * cmath.exp(1j * angle)
            F[u][v] = sum_c
    return F

In [5]:
#Perform DFT in the widthxheight region
def DFT2d_advanced(matrix, width, height):
    M = len(matrix)
    N = len(matrix[0])
    F = []
    for n in range(M):
        row = []
        for m in range(N):
            row.append(0j)
        F.append(row)
    
    for u in range((M - height) // 2, (M + height) // 2):
        for v in range((N - width) // 2, (N + width) // 2):
            sum_c = 0
            for x in range(M):
                for y in range(N):
                    angle = -2 * math.pi * ((u * x / M) + (v * y / N))
                    sum_c += matrix[x][y] * cmath.exp(1j * angle)
            F[u][v] = sum_c
    return F

In [6]:
#produce the vidualization
def magnitude_spectrum(F):
    M = len(F)
    N = len(F[0])
    
    spectrum = []
    for n in range(M):
        row = []
        for m in range(N):
            row.append(0)
        spectrum.append(row)

    for u in range(M):
        for v in range(N):
            if (math.log(1 + (abs(F[u][v]))) != 0):
                spectrum[u][v] = 255 / math.log(1 + (abs(F[u][v])))
            else:
                spectrum[u][v] = 0
    return spectrum

In [7]:
def inverseDFT2d(F):
    M = len(F)
    N = len(F[0])
    matrix = []
        
    for m in range(M):
        row = []
        for n in range(N):
            row.append(0)
        matrix.append(row)
        
    for x in range(M):
        for y in range(N):
            sum_c = 0
            for u in range(M):
                for v in range(N):
                    angle = 2 * math.pi * ((u * x / M) + (v * y / N))
                    sum_c += F[u][v] * cmath.exp(1j * angle)
            value = abs(sum_c) / (M * N)
            matrix[x][y] = int(min(max(value, 0), 255))
    return matrix

In [8]:
def inverseDFT2d_advanced(F, width, height):
    M = len(F)
    N = len(F[0])
    matrix = []
        
    for m in range(M):
        row = []
        for n in range(N):
            row.append(0j)
        matrix.append(row)

    for x in range(M):
        for y in range(N):
            sum_c = 0j
            for u in range((M - height) // 2, (M + height) // 2):
                for v in range((N - width) // 2, (N + width) // 2):
                    angle = 2 * math.pi * ((u * x / M) + (v * y / N))
                    sum_c += F[u][v] * cmath.exp(1j * angle)
            value = abs(sum_c) / (M * N)
            matrix[x][y] = int(min(max(value, 0), 255))
    return matrix

In [9]:
# applying a circular musk
def filter2d(F, lowBound, highBound):
    M = len(F)
    N = len(F[0])
    for x in range(M):
        for y in range(N):
            if (((x - M // 2)**2 + (y - N // 2)**2) < int(lowBound)**2) or (((x - M // 2)**2 + (y - N // 2)**2) > int(highBound)**2):
                F[x][y] = 0j
    return F

In [10]:
def process_image(img):
    
    matrix = image_to_matrix(img)

    F = DFT2d(matrix)
    spectrum = magnitude_spectrum(F)
    FreqSpectrum_img = matrix_to_image(spectrum)

    reconstructed = inverseDFT2d(F)
    recon_img = matrix_to_image(reconstructed)
    return FreqSpectrum_img, recon_img

In [11]:
#DFT advanced
def process_image_advanced(img, lowBound, highBound, width, height):
    
    matrix = image_to_matrix(img)
    orig_img = matrix_to_image(matrix)

    F = DFT2d_advanced(matrix, width, height)
    F = filter2d(F, lowBound, highBound)
    
    spectrum = magnitude_spectrum(F)
    FreqSpectrum_img = matrix_to_image(spectrum)

    reconstructed = inverseDFT2d_advanced(F, width, height)
    recon_img = matrix_to_image(reconstructed)
    return orig_img, FreqSpectrum_img, recon_img

In [12]:
#plot every pixel in the 3D space
def plotImage(img, sampleDown, elev, azim, darkColor):
    matrix = img.convert("RGB")

    arr = np.array(matrix)
    pixels = arr.reshape(-1, 3)

    pixels = pixels[::sampleDown]
    
    R = pixels[:, 0]
    G = pixels[:, 1]
    B = pixels[:, 2]
    
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111, projection = '3d')
    if (darkColor):
        ax.scatter(R, G, B, c='black', marker = 'o', s = 1, alpha = 0.6)
    else:
        ax.scatter(R, G, B, c = pixels / 255.0, marker = 'o', s = 1, alpha = 0.6)

    ax.set_xlabel('Red')
    ax.set_ylabel('Green')
    ax.set_zlabel('Blue')
    ax.set_xlim(0, 255)
    ax.set_ylim(0, 255)
    ax.set_zlim(0, 255)
    ax.set_title('Image pixels in RGB color space')
    
    ax.view_init(elev=elev, azim=azim)
    
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

In [13]:
#Mixing colors in the RGB colorspace by interpolating for the best combination of vectors.
def mix_colorsRGB(rt, gt, bt, pCount, r1, g1, b1, r2, g2, b2, r3, g3, b3, r4, g4, b4, r5, g5, b5):
    try:
        target_rgb = np.array([int(rt), int(gt), int(bt)]) / 255.0

        all_paint_rgbs = [
            [r1, g1, b1],
            [r2, g2, b2],
            [r3, g3, b3],
            [r4, g4, b4],
            [r5, g5, b5],
        ]

        paint_rgbs = []
        for i in range(int(pCount)):
            rgb = [int(all_paint_rgbs[i][j]) for j in range(3)]
            paint_rgbs.append(np.array(rgb) / 255.0)

        paint_rgbs = np.array(paint_rgbs)
        n = len(paint_rgbs)

        def objective(weights):
            mixed = np.dot(weights, paint_rgbs)
            return np.linalg.norm(mixed - target_rgb)

        constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
        bounds = [(0, 1)] * n
        initial_guess = np.ones(n) / n

        result = minimize(objective, initial_guess, bounds=bounds, constraints=constraints)

        if not result.success:
            return "Optimization failed. Try different input values.", None

        weights = result.x
        recipe = [
            f"{round(100 * weights[i])}% of {tuple(map(int, paint_rgbs[i] * 255))}"
            for i in range(n)
        ]
        mixed_rgb = np.dot(weights, paint_rgbs)
        mixed_rgb_255 = tuple(map(int, mixed_rgb * 255))

        text_output = f"Closest mix:\n" + "\n".join(recipe) + f"\n\nApproximate Resulting Color: {mixed_rgb_255}"

        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('Red')
        ax.set_ylabel('Green')
        ax.set_zlabel('Blue')
        ax.set_xlim(0, 255)
        ax.set_ylim(0, 255)
        ax.set_zlim(0, 255)
        
        for i, color in enumerate(paint_rgbs):
            ax.scatter(*color*255, color=color, s=100, label=f'Paint {i+1}')
        
        ax.scatter(*target_rgb*255, color=target_rgb, s=200, edgecolor='black', marker='X', label='Target')
        
        ax.legend(loc='upper left')
        
        buf = BytesIO()
        plt.tight_layout()
        plt.savefig(buf, format="png")
        buf.seek(0)
        plt.close()
        
        img = Image.open(buf)
        return text_output, img

    except Exception as e:
        return f"Error: {e}", None

In [14]:
#Mixing colors in the CIE colorspace (more realistic) by interpolating for the best combination of vectors.
def mix_colorsCIE(rt, gt, bt, pCount, r1, g1, b1, r2, g2, b2, r3, g3, b3, r4, g4, b4, r5, g5, b5):
    try:
        target_rgb = np.array([int(rt), int(gt), int(bt)]) / 255.0
        target_lab = rgb2lab(target_rgb.reshape(1, 1, 3)).reshape(3)

        all_paint_rgbs = [
            [r1, g1, b1],
            [r2, g2, b2],
            [r3, g3, b3],
            [r4, g4, b4],
            [r5, g5, b5],
        ]

        paint_rgbs = []
        for i in range(int(pCount)):
            rgb = [int(all_paint_rgbs[i][j]) for j in range(3)]
            paint_rgbs.append(np.array(rgb) / 255.0)

        paint_rgbs = np.array(paint_rgbs)
        n = len(paint_rgbs)

        paint_labs = np.array([
            rgb2lab(rgb.reshape(1, 1, 3)).reshape(3) for rgb in paint_rgbs
        ])

        def objective(weights):
            mixed_lab = np.dot(weights, paint_labs)
            return np.linalg.norm(mixed_lab - target_lab)

        constraints = {'type': 'eq', 'fun': lambda w: np.sum(w) - 1}
        bounds = [(0, 1)] * n
        initial_guess = np.ones(n) / n

        result = minimize(objective, initial_guess, bounds=bounds, constraints=constraints)

        if not result.success:
            return "Optimization failed. Try different input values.", None

        weights = result.x
        mixed_lab = np.dot(weights, paint_labs)
        mixed_rgb = lab2rgb(mixed_lab.reshape(1, 1, 3)).reshape(3)
        mixed_rgb_255 = tuple(map(int, np.clip(mixed_rgb * 255, 0, 255)))

        recipe = [
            f"{round(100 * weights[i])}% of {tuple(map(int, paint_rgbs[i] * 255))}"
            for i in range(n)
        ]

        text_output = f"Closest perceptual mix:\n" + "\n".join(recipe) + f"\n\nApproximate Resulting Color: {mixed_rgb_255}"

        # Plotting
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_xlabel('Red')
        ax.set_ylabel('Green')
        ax.set_zlabel('Blue')
        ax.set_xlim(0, 255)
        ax.set_ylim(0, 255)
        ax.set_zlim(0, 255)

        for i, rgb in enumerate(paint_rgbs):
            ax.scatter(*(rgb * 255), color=rgb, s=100, label=f'Paint {i+1}')

        ax.scatter(*(target_rgb * 255), color=target_rgb, s=200, edgecolor='black', marker='X', label='Target')

        ax.legend(loc='upper left')

        buf = BytesIO()
        plt.tight_layout()
        plt.savefig(buf, format="png")
        buf.seek(0)
        plt.close()

        img = Image.open(buf)
        return text_output, img

    except Exception as e:
        return f"Error: {e}", None

In [15]:
#based on the input, generate an output in either RGB space or CIE Lab colorspace
def mix_colors(CIE, rt, gt, bt, pCount, r1, g1, b1, r2, g2, b2, r3, g3, b3, r4, g4, b4, r5, g5, b5):
    if (CIE):
        return mix_colorsCIE(rt, gt, bt, pCount, r1, g1, b1, r2, g2, b2, r3, g3, b3, r4, g4, b4, r5, g5, b5)
    else:
        return mix_colorsRGB(rt, gt, bt, pCount, r1, g1, b1, r2, g2, b2, r3, g3, b3, r4, g4, b4, r5, g5, b5)

In [ ]:
#this entire section is the graphical interface
with gr.Blocks(title="Test") as GUI:
    with gr.Tab("Plotting an Image"):
        gr.Markdown("### What an Image Looks Like in the RGB Color Space")
        gr.Markdown("Input Your Image Here")

        with gr.Row():
            with gr.Column():
                input_image = gr.Image(type="pil", label="Upload")
                submit_btn = gr.Button("Process")
                sampleDown = gr.Slider(1, 50, value = 1,step=1, label="Sample Downsize", info="1 is original size. Increase to reduce lag.")
                elevation = gr.Slider(-90, 90, value = 30, label="Elevation in Degrees", info="Change the elevation of the view.")
                rotation = gr.Slider(0, 360, value = 45, label="Rotation in Degrees", info="Change the angle of the view.")
                inBlack = gr.Checkbox(label="Graph in Black and White")
            with gr.Column():
                scatterPlot = gr.Image(type = "pil", label = "The Image in Color Space")
        submit_btn.click(
            fn = plotImage,
            inputs = [input_image, sampleDown, elevation, rotation, inBlack],
            outputs = [scatterPlot]
        )
        
    sampleDown.change(fn=plotImage, inputs=[input_image, sampleDown, elevation, rotation, inBlack], outputs=[scatterPlot])
    elevation.change(fn=plotImage, inputs=[input_image, sampleDown, elevation, rotation, inBlack], outputs=[scatterPlot])
    rotation.change(fn=plotImage, inputs=[input_image, sampleDown, elevation, rotation, inBlack], outputs=[scatterPlot])
    inBlack.change(fn=plotImage, inputs=[input_image, sampleDown, elevation, rotation, inBlack], outputs=[scatterPlot])
    with gr.Tab("Color Mixer"):
        gr.Markdown("### Finding the Recipe of a Color")

        with gr.Row():
            with gr.Column(scale=1):
    
                btn = gr.Button("Calculate")
                CIE = gr.Checkbox(label="Use the CIE colorspace for realistic mixing.")
                
                gr.Markdown("### Target Color in RGB")
                with gr.Row():
                    rt = gr.Textbox(label="R", value="0")
                    gt = gr.Textbox(label="G", value="0")
                    bt = gr.Textbox(label="B", value="0")
    
                gr.Markdown("### Input Colors in RGB")
                pCount = gr.Slider(1,5, value = 1, step = 1, label="Number of Input Colors")
    
                with gr.Row():
                    r1 = gr.Textbox(label="R", value="0")
                    g1 = gr.Textbox(label="G", value="0")
                    b1 = gr.Textbox(label="B", value="0")
                with gr.Row():
                    r2 = gr.Textbox(label="R", value="0")
                    g2 = gr.Textbox(label="G", value="0")
                    b2 = gr.Textbox(label="B", value="0")
                with gr.Row():
                    r3 = gr.Textbox(label="R", value="0")
                    g3 = gr.Textbox(label="G", value="0")
                    b3 = gr.Textbox(label="B", value="0")
                with gr.Row():
                    r4 = gr.Textbox(label="R", value="0")
                    g4 = gr.Textbox(label="G", value="0")
                    b4 = gr.Textbox(label="B", value="0")
                with gr.Row():
                    r5 = gr.Textbox(label="R", value="0")
                    g5 = gr.Textbox(label="G", value="0")
                    b5 = gr.Textbox(label="B", value="0")
    
            with gr.Column(scale=1):
                output = gr.Textbox(label="Result", lines=8)
                mixGraph = gr.Image(label="RGB Color Plot")
        btn.click(mix_colors, inputs=[CIE, rt, gt, bt, pCount, r1, g1, b1, r2, g2, b2, r3, g3, b3, r4, g4, b4, r5, g5, b5], outputs=[output, mixGraph])
    with gr.Tab("Grayscale DFT"):
        gr.Markdown("### Grayscale Discrete Fourier Transform")
        gr.Markdown("A 40x40 pixel image takes ~3 seconds. Note the O^4 time complexity of DFT.")

        with gr.Row():
            with gr.Column():
                input_image = gr.Image(type="pil", label="Upload")
                submit_btn = gr.Button("Process")

            with gr.Column():
                output_magnitude = gr.Image(type = "pil", label = "Magnitude Spectrum").style(height=240)
                output_reconstructed = gr.Image(type = "pil", label = "Reconstructed Image").style(height=240)

        submit_btn.click(
            fn=lambda img: process_image(img),
            inputs=input_image,
            outputs=[output_magnitude, output_reconstructed]
        )

    with gr.Tab("Grayscale DFT Filter"):
        gr.Markdown("### Applying Filters with Discrete Fourier Transform ")
        gr.Markdown("A 40x40 pixel image takes ~3 seconds. Note the O^4 time complexity of DFT.")

        with gr.Row():
            with gr.Column():
                input_image_adv = gr.Image(type = "pil", label = "Upload")
                lowBound = gr.Textbox(label = "Radius of the Lower Bound (Pixels)", lines = 1, value = 0)
                highBound = gr.Textbox(label = "Radius of the Upper Bound (Pixels)", lines = 1, value = 32767)
                width = gr.Slider(1, 100, value = 100,step=1, label="width", info="Width of the DFT Range")
                height = gr.Slider(1, 100, value = 100,step=1, label="height", info="Length of the DFT Range")
                submit_btn_adv = gr.Button("Process")
            
            with gr.Column():
                output_original_adv = gr.Image(type = "pil", label = "Original in Grayscale").style(height=240)
                output_magnitude_adv = gr.Image(type = "pil", label = "Magnitude Spectrum").style(height=240)
                output_reconstructed_adv = gr.Image(type = "pil", label = "Reconstructed Image").style(height=240)
        
        submit_btn_adv.click(
            fn = process_image_advanced,
            inputs = [input_image_adv, lowBound, highBound, width, height],
            outputs = [output_original_adv, output_magnitude_adv, output_reconstructed_adv]
        )

In [ ]:
GUI.launch()